In [1]:
import numpy as np

# Patriot missile system problem

The patriot missile clock system stores a 24-bit binary representation of time intervals of 0.1s. Without two's complement, where the leftmost bit is used to store the negative number, the missile system uses 8 bits to represent the decimal equivalent of 0.1.

In [48]:
def decimal_binary_estimator(number, n_bits, func=binary_calculator):
    ''' Takes a fractional number and returns its binary approximation 
    based on 8-bit truncation error, along with the actual fractional number estimated by
    the 8-bit digits. Any number requiring less than 8 bits will be exact.
    
    Inputs
    -----
    number: float -- The fractional number to be represented
    n_bits: int -- The number of bits to hold the binary representation
    func: function -- Default is the function binary_calculator, which carries out the 
    computation of the the estimate of the number base on an n bit storage of the fractional
    part.
    
    Outputs
    -------
    representation: string -- A string of the binary representation of the input number. 
    result: The estimate of the input based on n bits. Any numbers requiring less than n bits
    to store will be exact.
    '''
    func = binary_calculator
    frac = .5    # seed to allow the while loop to begin
    digits = []
    i = 1     # counter, which will stop at n_bits 
    while (frac != 0.) & (i <= n_bits):
        num = number*2
        integer = int(num)
        #print(f'num: {num}, integer: {integer}, frac: {frac}')
        frac = num % 1
        digits.append(integer)
        number = frac
        i += 1
    result = func(digits)
    binary_rep = [str(x) for x in digits]
    representation = '.'+''.join(binary_rep)
    return representation, result


In [30]:
def binary_calculator(digits):
    total = 0
    for idx, exp in enumerate(digits):
        if exp == 1:
            total += 2**(-(idx+1))
        else:
            continue
    return total

In [51]:
decimal_binary_estimator(0.1, 24)

('.000110011001100110011001', 0.09999996423721313)

In [21]:
digits

[1, 0, 1]

In [25]:
total = 0
for idx, exp in enumerate(digits):
    if exp == 1:
        total += 2**(-(idx+1))
    else:
        continue

In [32]:
digits = [1, 0, 1]

In [35]:
new_digits = [str(x) for x in digits]

In [36]:
'.' + ''.join(new_digits)

'.101'